In [23]:
import keras
import numpy as np
import os
from pathlib import Path
from keras import layers
import random
import sys

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')

results_dir.mkdir(parents=True, exist_ok=True)

output_dir = current_dir.joinpath('output')
output_dir.mkdir(parents = True, exist_ok = True)

corpus = current_dir.joinpath('hobbit.txt')

text = open(corpus).read().lower()
print('Corpus length:', len(text))


Corpus length: 516229


In [24]:
maxlen = 60

step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 172057
Unique characters: 58
Vectorization...


In [25]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

In [26]:
optimizer = keras.optimizers.RMSprop(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

In [27]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [28]:
for epoch in range(1, 61):
        print('epoch', epoch)
        model.fit(x, y, batch_size = 128, epochs = 1)
        start_index = random.randint(0, len(text) - maxlen - 1)
        generated_text = text[start_index: start_index + maxlen]
        start_text = generated_text
        output_text = 'Initial seed: ' + generated_text
        print('--- Generating with seed: "' + generated_text + '"')
        
        for temperature in [0.2, 0.5, 1.0, 1.2]:
            print('------ temperature:', temperature)
            output_text += '\ntemperature: ' + str(temperature) + '\n' + start_text
            sys.stdout.write(start_text)
            generated_text = start_text
        
            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.
            
                preds = model.predict(sampled, verbose = 0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]
            
                generated_text += next_char
                output_text += next_char
                generated_text = generated_text[1:]
            
                sys.stdout.write(next_char)
        output_path = results_dir.joinpath('training.' + str(epoch) + '.txt')
        output_file = open(output_path, "w")
        output_file.write(output_text)
        output_file.close()

epoch 1
1345/1345 [==============================] - 289s 215ms/step - loss: 1.8813
--- Generating with seed: "tack them also from behind and above; but there was no time "
------ temperature: 0.2
tack them also from behind and above; but there was no time the mountains and the mountains the was and the mountains the rid the great the had 
stare the mountains the mountains and the mountains and the mountains was all the 
was the 
was the was a 
















































































































































































































------ temperature: 0.5
tack them also from behind and above; but there was no time the has starked the 
read the dragon in the mountains was a shore hold in the trees wolld the share and the mountains the river 
the goblins was and the 
very alrought the down and the son’t 
came the 
warves the mountains the had of stil the wood be for the 

with lioking thinur they rest innt of the path liff obbit. novies with yor, 
and seemed to 
near. left over hissing------ temperature: 1.2
, 
but gollum thought it was a riddle, and he was frightfully end 
headneds, i feelly or a like sundarhm up of known tal settils.” 

a longe i be streud. and into the hose away, “but even, and you u lut all. 
“clapt you were is. very did ?, 
these only it as i will fage. my sown yee?” remandice, if it flowed it was 
pine shalluguint, and he rewired, these mightroiced and truuthqusaned. green that 
experted it ikoned. somechit the imairins, before he had diepoch 9
1345/1345 [==============================] - 289s 215ms/step - loss: 1.2862
--- Generating with seed: "“guess 
again!” 

“s-s-s-s-s,” said gollum more upset than e"
------ temperature: 0.2
“guess 
again!” 

“s-s-s-s-s,” said gollum more upset than evenice the 
was he was to the trees and was the forest the 
mountains; and they were all the dragon of the 
mountains; and the dark was the drag

things, and they had not the passes and stood with the winds. where the 
sweet of the dark was it was a peace of can anywhered to the 
while the remember and he could not feel the sent way that he 
shoped that they were brought before he had thought a great heart. the 
stream was ------ temperature: 1.0
way to the east, the terrible forest of mirkwood. 

the dwarves began to that began us block and bagginves and ever in one 
all their measching feeling as bloyone his fremrirevery, but now caught it must follow, and the 
unnexcook must say forp-stidin fow burntwery thoin, and sluch he would their cobbit, and they 
looked on to daik. “so as i am 
seoke?” said yome fet to the salt; and they came the found the 
uncomcany fant. they lives iudwards at the courd------ temperature: 1.2
way to the east, the terrible forest of mirkwood. 

the dwarves. 

it only unbushearly repeabred thorin at all. 

“a arcarled thorin, pastacerhars, for his house islive it. he 
grey remem which akeeside rvered h

<ipython-input-27-462ec6224322>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


n, and he could not like the 
lake. they were ------ temperature: 0.5
st enemies, when they captured them. the giant 
spiders were all the dragon had been hand into the entrance. 



all the passage was the spiders to go to the only perhoped of 
the spider. 

“are why light with the wood-elving the side of the mountain, 
and the side of the glams of the green head and they saw his head 
hole of the captions behind them and in the eastern flap stones 
and the easters and they saw his side and his head. 

“something its,” said------ temperature: 1.0
st enemies, when they captured them. the giant 
spiders were never dain, 
and gollum did not don’t get it, if you will stars remained to come along the 
south, before, still still, and swords for began on the nene from answer, and they lix plain and really not 
he heard thorin with something, 


pnockow, which will don’t don’t amiance it. nor they been came for it. i big 
fell into the dwarves, by through goblinst. which is it everythen the e

1345/1345 [==============================] - 286s 212ms/step - loss: 1.1726
--- Generating with seed: " either 
he should have gone back quietly and warned his fri"
------ temperature: 0.2
 either 
he should have gone back quietly and warned his friends and 
best the strong hobbit was some of the dwarves and the dragon 
small to the stores of the stores and the shadow of the walls 
of the dark who had been the tree of the dragon and the dragon 
small far and the dragon and the dwarves and he could not see the 
stream of the dark who had been the way or have to the light of the 
mountains, and then he was a drink of the dragon for the dwarves------ temperature: 0.5
 either 
he should have gone back quietly and warned his friends in the 
mountains, and soon were miserading to go back for we should 
the tunnels and so they were alf the edge of the secret of the 
wood- houstialeps and the magic or in the north of the woods 




































































the swires they------ temperature: 1.2
spells, 
while hammers fell like ringing bells 
in places deept too gear.” 
he sat down the anksided green trolls, stual; but so thorin intleking ment. “wish laugh he esoited. “with to foll bettlls arud unchostle; and it was to different busy.d, 
asy dragoning and grasved the while ea’htion of the lands of the limpee 
that of the rooled you amfurtlable letted, if he was he bright the ineve-yet way egs, years so-hojojoy the parnal bopt bent ony or 
anscrifvepoch 37
1345/1345 [==============================] - 286s 213ms/step - loss: 1.1501
--- Generating with seed: "came in rolling round drum-shaped sections of logs, 
smoothe"
------ temperature: 0.2
came in rolling round drum-shaped sections of logs, 
smoother to the strong of the stream of the dragon and the dark 



























































































































































































been feeling along and the course that was not a feeling of the 
an------ temperature: 1.0
r and make in the opposite direction. to the left there, that they turned in a huge ban in the 
south and elvish open and out of tomfed to his eyous back and they crief 
suppering, and first they had sen him foodk they . you with 
easy averoh, much of the others, and his long open small has even and he 



wit green, shangan ceir your feechessing to lake. as that plained them. it! lost in pocket. 

bilbo sat in far were right and dales, 
the dwarves thinki------ temperature: 1.2
r and make in the opposite direction. to the left there, that it, 
only water of the hissing, killed as on dwarvist. shotting grey-most in answere. theyooses and merlen upon the gramm. 

ping luck he heard himself, casled dim 
on the gove in the goblins upon ob;dyburselves. i they was nast mown 
cate! bed clations “latch! why there went. nopening used it!” 


s, opents in the end. bilbo has calmed, i shall could fieren br

stand of the mountain, and the was sudden; a little shore in the 
stream and the dark had standing the mountain and the dragon 
and the ------ temperature: 0.5
 your pardon, i haven’t asked for anything!” 

“yes, you have no the river,” said the passage they were still from the 
spiders, and i should not think i all more since the grey of the stone of light, and they had no 
beat, and he could not going to the bridges, and they are sometimes. 

“i gave a hold before you can the trees, and the mountain!” said 
the wood-elvisu songs their cold and the men of the leaves were still and the 
dark snoring the fire were------ temperature: 1.0
 your pardon, i haven’t asked for anything!” 

“yes, you have stord!” he is yet need of a butler of larges and liest, and the 
sumely was getting very your beard there poniessmenteddy contrils 
all him on evening, but murder! 
the flown. “treasting you acteated. i you tunned aon, ord wherever they came from 
intlew kecifort and bigger first and find the 

1345/1345 [==============================] - 323s 240ms/step - loss: 1.1113
--- Generating with seed: " 
as mountain-streams do of a summer evening, when sun has b"
------ temperature: 0.2
 
as mountain-streams do of a summer evening, when sun has been 
to the spiders, and they saw the trees and strongs and the master 
would like to the spiders to the same and stumbled and sturning 
at the spiders, and the dwarves had been to the way of the mountains. the 
was soon as the dwarves had the dragon before he was something 
and splash and spiders and the spiders and struck and the began 
to be seem, and the saw the spiders behind the spiders ston------ temperature: 0.5
 
as mountain-streams do of a summer evening, when sun has been to the 
river and the stars of the strang more they turned with the 
forthance. 

“thind it seems to feet be sitting off, and little to the great 
ofthing stoness, and soon had soon thought the hobbit was though his eath but 
the misty mountains, and could starte

In [30]:


for attempt in range(1, 21):
        print('attempt', attempt)
        start_index = random.randint(0, len(text) - maxlen - 1)
        generated_text = text[start_index: start_index + maxlen]
        start_text = generated_text
        output_text = 'Initial seed: ' + generated_text
        print('--- Generating with seed: "' + generated_text + '"')
        
        for temperature in [0.2, 0.5, 1.0, 1.2]:
            print('------ temperature:', temperature)
            output_text += '\ntemperature: ' + str(temperature) + '\n' + start_text
            sys.stdout.write(start_text)
            generated_text = start_text
         
            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.
            
                preds = model.predict(sampled, verbose = 0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]
            
                generated_text += next_char
                output_text += next_char
                generated_text = generated_text[1:]
            
                sys.stdout.write(next_char)
        output_path = output_dir.joinpath('output.' + str(attempt) + '.txt')
        output_file = open(output_path, "w")
        output_file.write(output_text)
        output_file.close()

attempt 1
--- Generating with seed: "t. the sparks were burning holes in the goblins, and the 
sm"
------ temperature: 0.2
t. the sparks were burning holes in the goblins, and the 
small south and the master of the great ray and the same his 
words and the trees and splash and the spiders still they came 
the spiders a

<ipython-input-27-462ec6224322>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


nd stumbled to the spiders story and the dwarves 
and the spiders and stumbled and the great flate of the strosh. 

“we shall see the south of the great goblin, and the hall of the 
storm of well over the spiders and the spiders slider, and they 
saw the way of t------ temperature: 0.5
t. the sparks were burning holes in the goblins, and the 
smosh what the dwarves were to the main fathers all in a little 
flater the last of the worst of the flathing of the battle before the 
face of the green spear of the great carrock and so the passages 
in the river end and green them rather head you alls. the was suddenly with 
it. there is to be he had do as he was the saw the forth was 
walls and the face of the blust trees, and he thought the oth------ temperature: 1.0
t. the sparks were burning holes in the goblins, and the 
smose a good bears, or not be help in the great race of untillumely on 
them. bilbo had most frai last more about with a face, and then 
things uppencing across the empty 

saw the spiders small glint of the mountains, and then the hobbit 
was something to the way of the breath and stumbled to the trees 
with a little share of the mounta------ temperature: 0.5
 kettle just beginning to sing!” it was 
not the last time to see, a bist of the rows of the stream and 
the head and the master can now song at the magic him soon and 
beside the great father, and had expected and the next time they 
was a good stant than the great black bars, and there is to 
tell up and sat it, and standing as bilbo have been the master 
way at last, and soon his head was waiting into the side.” 

“going again,” said gandalf he could be------ temperature: 1.0
 kettle just beginning to sing!” it was 
not the last time true-shares and a dire cutly open animap of fire, and splelved, eftening 
morning bombur. he are some muching. there he hached at the 
houtb?” they defrocked him from the dwarves had burned: it was tell 
are, thing great days; but then the dark all we are plad, sow f

world the master which the river were all the began to see the 
spiders to the hobbit. 

“thind you trunk at the goblin,” said bil------ temperature: 0.5
 and fought like mad, as dwarves will when cornered. 

thorin was like the mountains. the great most was the began the 
way out of smaughts and the spiders caught and the of the last 

















our streams and as well the began the trolls, and it was the 
course of the side of the treast of the catch; but they thought he 
was in the wish they were fine that they were can all stars, 
smash the began to think to the way as for the master to see the 
m------ temperature: 1.0
 and fought like mad, as dwarves will when cornered. 

thorin had feet that made bilbo askwmenly by. then it had there gettinave you be dwarf. there jumped to bey any fit little line. 

then it made part of the mountain, and though bilbo laid it, 
and we seef part of rist and sverry, passedling on in mireg. gandalf 
with their left him working thorin shilleg?”